In [2]:
import pandas as pd
import numpy as np
from tensorflow import keras
from math import sqrt
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize as st

In [14]:
def tf_idf(text):
    sentences = st(text)
    vect = TfidfVectorizer(min_df=1)
    tfidf = vect.fit_transform(sentences)
    matrix = np.asarray((tfidf * tfidf.T).A)
    matrix.resize(5, 5)
    return matrix.flatten()

In [15]:
def flat_input(corpus, classes):
    docs = [title + '. ' + text for title, text in corpus]
    prep = [tf_idf(text) for text in docs]
    outp = [[1, 0] if i == 'FAKE' else [0, 1] for i in classes]
    return np.asarray(prep), np.asarray(outp)

In [14]:
def max_pos(l):
    m = 0
    j = 0
    for i, n in enumerate(l):
        if n > m:
            j = i
            m = n
    return j

In [12]:
def calculate_metrics(pred, real):
    result = []
    for p,r in zip(pred, real):
        (i, j) = (max_pos(p), max_pos(r))
        if i == j and r == [1,0]:
            result.append('TP')
        elif i == j and r == [0,1]:
            result.append('TN')
        elif i != j and r == [1,0]:
            result.append('FN')
        elif i != j and r == [0,1]:
            result.append('FP')
    precision = result.count('TP') / (result.count('TP') + result.count('FP'))
    recall = result.count('TP') / (result.count('TP') + result.count('FN'))
    f1 = 2*((precision*recall)/(precision + recall))
    accuracy = (result.count('TP') + result.count('TN')) / (result.count('TP') + result.count('TN') + result.count('FN') + result.count('FP'))
    msg = 'Precision: \t' + str(precision) + '\t --> \t' + str(int(precision * 100)) + '%\n'
    msg += 'Recall: \t' + str(recall) + '\t --> \t' + str(int(recall * 100)) + '%\n'
    msg += 'F1: \t\t' + str(f1) + '\t --> \t' + str(int(f1 * 100)) + '%\n'
    msg += 'Accuracy: \t' + str(accuracy) + '\t --> \t' + str(int(accuracy * 100)) + '%'
    print(msg)

In [5]:
cols = ['ID', 'TITLE', 'TEXT', 'LABEL']
data = pd.read_csv("fake_or_real_news.csv", names=cols, header=0)

In [19]:
k = len(data[cols[3]]) // 2
x_train, y_train = flat_input(list(zip(data[cols[1]], data[cols[2]]))[:k], data[cols[3]][:k])
x_validate, y_validate = flat_input(list(zip(data[cols[1]], data[cols[2]]))[k:], data[cols[3]][k:])

In [103]:
model = keras.Sequential()
model.add(keras.layers.Dense(units=500, activation='relu', input_dim=25))
model.add(keras.layers.Dense(units=200, activation='relu'))
model.add(keras.layers.Dense(units=50, activation='relu'))
model.add(keras.layers.Dense(units=2, activation='softmax'))

In [104]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

In [110]:
model.fit(x_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
3167/3167 [==============================] - 0s 67us/step - loss: 0.5799 - acc: 0.7079
Epoch 2/20
3167/3167 [==============================] - 0s 61us/step - loss: 0.5770 - acc: 0.7117
Epoch 3/20
3167/3167 [==============================] - 0s 59us/step - loss: 0.5760 - acc: 0.7092
Epoch 4/20
3167/3167 [==============================] - 0s 60us/step - loss: 0.5739 - acc: 0.7098
Epoch 5/20
3167/3167 [==============================] - 0s 61us/step - loss: 0.5702 - acc: 0.7152
Epoch 6/20
3167/3167 [==============================] - 0s 62us/step - loss: 0.5691 - acc: 0.7123
Epoch 7/20
3167/3167 [==============================] - 0s 64us/step - loss: 0.5694 - acc: 0.7183
Epoch 8/20
3167/3167 [==============================] - 0s 60us/step - loss: 0.5671 - acc: 0.7168
Epoch 9/20
3167/3167 [==============================] - 0s 61us/step - loss: 0.5677 - acc: 0.7171
Epoch 10/20
3167/3167 [==============================] - 0s 63us/step - loss: 0.5671 - acc: 0.7168
Epoch 11/20
3167/31

In [111]:
model.evaluate(x_validate, y_validate, batch_size=32)

3168/3168 [==============================] - 0s 22us/step


[0.5584360550750386, 0.7190656565656566]

In [112]:
result = model.predict(x_validate, batch_size=32).tolist()

In [113]:
c = data[cols[3]].tolist()[:k]
str(c.count('FAKE')) + ' / ' + str(c.count('REAL'))

'1601 / 1566'

In [114]:
calculate_metrics(result, y_validate.tolist())

Precision: 	0.7227001985440106	 --> 	72%
Recall: 	0.6986564299424184	 --> 	69%
F1: 		0.7104749512036433	 --> 	71%
Accuracy: 	0.7190656565656566	 --> 	71%


In [3]:
from feature_extraction import manager

In [48]:
k = len(data[cols[3]]) // 2

x = list(zip(data[cols[1]], data[cols[2]]))
y = data[cols[3]]
dim = 5

In [49]:
x_train, y_train = manager.flat_input(x[:k], y[:k], dim)
x_validate, y_validate = manager.flat_input(x[k:], y[k:], dim)

Processed 0 3167
Processed 1 3167
Processed 2 3167
Processed 3 3167
Processed 4 3167
Processed 5 3167
Processed 6 3167
Processed 7 3167
Processed 8 3167
Processed 9 3167
Processed 10 3167
Processed 11 3167
Processed 12 3167
Processed 13 3167
Processed 14 3167
Processed 15 3167
Processed 16 3167
Processed 17 3167
Processed 18 3167
Processed 19 3167
Processed 20 3167
Processed 21 3167
Processed 22 3167
Processed 23 3167
Processed 24 3167
Processed 25 3167
Processed 26 3167
Processed 27 3167
Processed 28 3167
Processed 29 3167
Processed 30 3167
Processed 31 3167
Processed 32 3167
Processed 33 3167
Processed 34 3167
Processed 35 3167
Processed 36 3167
Processed 37 3167
Processed 38 3167
Processed 39 3167
Processed 40 3167
Processed 41 3167
Processed 42 3167
Processed 43 3167
Processed 44 3167
Processed 45 3167
Processed 46 3167
Processed 47 3167
Processed 48 3167
Processed 49 3167
Processed 50 3167
Processed 51 3167
Processed 52 3167
Processed 53 3167
Processed 54 3167
Processed 55 3167
Pr

Processed 437 3167
Processed 438 3167
Processed 439 3167
Processed 440 3167
Processed 441 3167
Processed 442 3167
Processed 443 3167
Processed 444 3167
Processed 445 3167
Processed 446 3167
Processed 447 3167
Processed 448 3167
Processed 449 3167
Processed 450 3167
Processed 451 3167
Processed 452 3167
Processed 453 3167
Processed 454 3167
Processed 455 3167
Processed 456 3167
Processed 457 3167
Processed 458 3167
Processed 459 3167
Processed 460 3167
Processed 461 3167
Processed 462 3167
Processed 463 3167
Processed 464 3167
Processed 465 3167
Processed 466 3167
Processed 467 3167
Processed 468 3167
Processed 469 3167
Processed 470 3167
Processed 471 3167
Processed 472 3167
Processed 473 3167
Processed 474 3167
Processed 475 3167
Processed 476 3167
Processed 477 3167
Processed 478 3167
Processed 479 3167
Processed 480 3167
Processed 481 3167
Processed 482 3167
Processed 483 3167
Processed 484 3167
Processed 485 3167
Processed 486 3167
Processed 487 3167
Processed 488 3167
Processed 48

Processed 870 3167
Processed 871 3167
Processed 872 3167
Processed 873 3167
Processed 874 3167
Processed 875 3167
Processed 876 3167
Processed 877 3167
Processed 878 3167
Processed 879 3167
Processed 880 3167
Processed 881 3167
Processed 882 3167
Processed 883 3167
Processed 884 3167
Processed 885 3167
Processed 886 3167
Processed 887 3167
Processed 888 3167
Processed 889 3167
Processed 890 3167
Processed 891 3167
Processed 892 3167
Processed 893 3167
Processed 894 3167
Processed 895 3167
Processed 896 3167
Processed 897 3167
Processed 898 3167
Processed 899 3167
Processed 900 3167
Processed 901 3167
Processed 902 3167
Processed 903 3167
Processed 904 3167
Processed 905 3167
Processed 906 3167
Processed 907 3167
Processed 908 3167
Processed 909 3167
Processed 910 3167
Processed 911 3167
Processed 912 3167
Processed 913 3167
Processed 914 3167
Processed 915 3167
Processed 916 3167
Processed 917 3167
Processed 918 3167
Processed 919 3167
Processed 920 3167
Processed 921 3167
Processed 92

Processed 1288 3167
Processed 1289 3167
Processed 1290 3167
Processed 1291 3167
Processed 1292 3167
Processed 1293 3167
Processed 1294 3167
Processed 1295 3167
Processed 1296 3167
Processed 1297 3167
Processed 1298 3167
Processed 1299 3167
Processed 1300 3167
Processed 1301 3167
Processed 1302 3167
Processed 1303 3167
Processed 1304 3167
Processed 1305 3167
Processed 1306 3167
Processed 1307 3167
Processed 1308 3167
Processed 1309 3167
Processed 1310 3167
Processed 1311 3167
Processed 1312 3167
Processed 1313 3167
Processed 1314 3167
Processed 1315 3167
Processed 1316 3167
Processed 1317 3167
Processed 1318 3167
Processed 1319 3167
Processed 1320 3167
Processed 1321 3167
Processed 1322 3167
Processed 1323 3167
Processed 1324 3167
Processed 1325 3167
Processed 1326 3167
Processed 1327 3167
Processed 1328 3167
Processed 1329 3167
Processed 1330 3167
Processed 1331 3167
Processed 1332 3167
Processed 1333 3167
Processed 1334 3167
Processed 1335 3167
Processed 1336 3167
Processed 1337 3167


Processed 1698 3167
Processed 1699 3167
Processed 1700 3167
Processed 1701 3167
Processed 1702 3167
Processed 1703 3167
Processed 1704 3167
Processed 1705 3167
Processed 1706 3167
Processed 1707 3167
Processed 1708 3167
Processed 1709 3167
Processed 1710 3167
Processed 1711 3167
Processed 1712 3167
Processed 1713 3167
Processed 1714 3167
Processed 1715 3167
Processed 1716 3167
Processed 1717 3167
Processed 1718 3167
Processed 1719 3167
Processed 1720 3167
Processed 1721 3167
Processed 1722 3167
Processed 1723 3167
Processed 1724 3167
Processed 1725 3167
Processed 1726 3167
Processed 1727 3167
Processed 1728 3167
Processed 1729 3167
Processed 1730 3167
Processed 1731 3167
Processed 1732 3167
Processed 1733 3167
Processed 1734 3167
Processed 1735 3167
Processed 1736 3167
Processed 1737 3167
Processed 1738 3167
Processed 1739 3167
Processed 1740 3167
Processed 1741 3167
Processed 1742 3167
Processed 1743 3167
Processed 1744 3167
Processed 1745 3167
Processed 1746 3167
Processed 1747 3167


...

Processed 1463 3168
Processed 1464 3168
Processed 1465 3168
Processed 1466 3168
Processed 1467 3168
Processed 1468 3168
Processed 1469 3168
Processed 1470 3168
Processed 1471 3168
Processed 1472 3168
Processed 1473 3168
Processed 1474 3168
Processed 1475 3168
Processed 1476 3168
Processed 1477 3168
Processed 1478 3168
Processed 1479 3168
Processed 1480 3168
Processed 1481 3168
Processed 1482 3168
Processed 1483 3168
Processed 1484 3168
Processed 1485 3168
Processed 1486 3168
Processed 1487 3168
Processed 1488 3168
Processed 1489 3168
Processed 1490 3168
Processed 1491 3168
Processed 1492 3168
Processed 1493 3168
Processed 1494 3168
Processed 1495 3168
Processed 1496 3168
Processed 1497 3168
Processed 1498 3168
Processed 1499 3168
Processed 1500 3168
Processed 1501 3168
Processed 1502 3168
Processed 1503 3168
Processed 1504 3168
Processed 1505 3168
Processed 1506 3168
Processed 1507 3168
Processed 1508 3168
Processed 1509 3168
Processed 1510 3168
Processed 1511 3168
Processed 1512 3168


Processed 1875 3168
Processed 1876 3168
Processed 1877 3168
Processed 1878 3168
Processed 1879 3168
Processed 1880 3168
Processed 1881 3168
Processed 1882 3168
Processed 1883 3168
Processed 1884 3168
Processed 1885 3168
Processed 1886 3168
Processed 1887 3168
Processed 1888 3168
Processed 1889 3168
Processed 1890 3168
Processed 1891 3168
Processed 1892 3168
Processed 1893 3168
Processed 1894 3168
Processed 1895 3168
Processed 1896 3168
Processed 1897 3168
Processed 1898 3168
Processed 1899 3168
Processed 1900 3168
Processed 1901 3168
Processed 1902 3168
Processed 1903 3168
Processed 1904 3168
Processed 1905 3168
Processed 1906 3168
Processed 1907 3168
Processed 1908 3168
Processed 1909 3168
Processed 1910 3168
Processed 1911 3168
Processed 1912 3168
Processed 1913 3168
Processed 1914 3168
Processed 1915 3168
Processed 1916 3168
Processed 1917 3168
Processed 1918 3168
Processed 1919 3168
Processed 1920 3168
Processed 1921 3168
Processed 1922 3168
Processed 1923 3168
Processed 1924 3168


Processed 2288 3168
Processed 2289 3168
Processed 2290 3168
Processed 2291 3168
Processed 2292 3168
Processed 2293 3168
Processed 2294 3168
Processed 2295 3168
Processed 2296 3168
Processed 2297 3168
Processed 2298 3168
Processed 2299 3168
Processed 2300 3168
Processed 2301 3168
Processed 2302 3168
Processed 2303 3168
Processed 2304 3168
Processed 2305 3168
Processed 2306 3168
Processed 2307 3168
Processed 2308 3168
Processed 2309 3168
Processed 2310 3168
Processed 2311 3168
Processed 2312 3168
Processed 2313 3168
Processed 2314 3168
Processed 2315 3168
Processed 2316 3168
Processed 2317 3168
Processed 2318 3168
Processed 2319 3168
Processed 2320 3168
Processed 2321 3168
Processed 2322 3168
Processed 2323 3168
Processed 2324 3168
Processed 2325 3168
Processed 2326 3168
Processed 2327 3168
Processed 2328 3168
Processed 2329 3168
Processed 2330 3168
Processed 2331 3168
Processed 2332 3168
Processed 2333 3168
Processed 2334 3168
Processed 2335 3168
Processed 2336 3168
Processed 2337 3168


Processed 2702 3168
Processed 2703 3168
Processed 2704 3168
Processed 2705 3168
Processed 2706 3168
Processed 2707 3168
Processed 2708 3168
Processed 2709 3168
Processed 2710 3168
Processed 2711 3168
Processed 2712 3168
Processed 2713 3168
Processed 2714 3168
Processed 2715 3168
Processed 2716 3168
Processed 2717 3168
Processed 2718 3168
Processed 2719 3168
Processed 2720 3168
Processed 2721 3168
Processed 2722 3168
Processed 2723 3168
Processed 2724 3168
Processed 2725 3168
Processed 2726 3168
Processed 2727 3168
Processed 2728 3168
Processed 2729 3168
Processed 2730 3168
Processed 2731 3168
Processed 2732 3168
Processed 2733 3168
Processed 2734 3168
Processed 2735 3168
Processed 2736 3168
Processed 2737 3168
Processed 2738 3168
Processed 2739 3168
Processed 2740 3168
Processed 2741 3168
Processed 2742 3168
Processed 2743 3168
Processed 2744 3168
Processed 2745 3168
Processed 2746 3168
Processed 2747 3168
Processed 2748 3168
Processed 2749 3168
Processed 2750 3168
Processed 2751 3168


Processed 3115 3168
Processed 3116 3168
Processed 3117 3168
Processed 3118 3168
Processed 3119 3168
Processed 3120 3168
Processed 3121 3168
Processed 3122 3168
Processed 3123 3168
Processed 3124 3168
Processed 3125 3168
Processed 3126 3168
Processed 3127 3168
Processed 3128 3168
Processed 3129 3168
Processed 3130 3168
Processed 3131 3168
Processed 3132 3168
Processed 3133 3168
Processed 3134 3168
Processed 3135 3168
Processed 3136 3168
Processed 3137 3168
Processed 3138 3168
Processed 3139 3168
Processed 3140 3168
Processed 3141 3168
Processed 3142 3168
Processed 3143 3168
Processed 3144 3168
Processed 3145 3168
Processed 3146 3168
Processed 3147 3168
Processed 3148 3168
Processed 3149 3168
Processed 3150 3168
Processed 3151 3168
Processed 3152 3168
Processed 3153 3168
Processed 3154 3168
Processed 3155 3168
Processed 3156 3168
Processed 3157 3168
Processed 3158 3168
Processed 3159 3168
Processed 3160 3168
Processed 3161 3168
Processed 3162 3168
Processed 3163 3168
Processed 3164 3168


In [55]:
model = keras.Sequential()
model.add(keras.layers.Dense(units=500, activation='relu', input_dim=dim**2))
model.add(keras.layers.Dense(units=200, activation='relu'))
model.add(keras.layers.Dense(units=50, activation='relu'))
model.add(keras.layers.Dense(units=2, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

In [56]:
model.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
3167/3167 [==============================] - 1s 231us/step - loss: 0.6831 - acc: 0.5500
Epoch 2/10
3167/3167 [==============================] - 0s 64us/step - loss: 0.6517 - acc: 0.6350
Epoch 3/10
3167/3167 [==============================] - 0s 63us/step - loss: 0.6219 - acc: 0.6549
Epoch 4/10
3167/3167 [==============================] - 0s 62us/step - loss: 0.6108 - acc: 0.6748
Epoch 5/10
3167/3167 [==============================] - 0s 64us/step - loss: 0.6034 - acc: 0.6763
Epoch 6/10
3167/3167 [==============================] - 0s 62us/step - loss: 0.5958 - acc: 0.7022
Epoch 7/10
3167/3167 [==============================] - 0s 63us/step - loss: 0.5908 - acc: 0.6959
Epoch 8/10
3167/3167 [==============================] - 0s 63us/step - loss: 0.5884 - acc: 0.6934
Epoch 9/10
3167/3167 [==============================] - 0s 63us/step - loss: 0.5828 - acc: 0.7057
Epoch 10/10
3167/3167 [==============================] - 0s 64us/step - loss: 0.5808 - acc: 0.7032


In [57]:
model.evaluate(x_validate, y_validate, batch_size=32)

3168/3168 [==============================] - 0s 74us/step


[0.5854969382888139, 0.6966540404040404]

In [58]:
result = model.predict(x_validate, batch_size=32).tolist()
calculate_metrics(result, y_validate.tolist())

Precision: 	0.7137784090909091	 --> 	71%
Recall: 	0.6429942418426103	 --> 	64%
F1: 		0.6765398855604173	 --> 	67%
Accuracy: 	0.6966540404040404	 --> 	69%


In [59]:
[i for i in zip(result, y_validate.tolist())]

[([0.2892616391181946, 0.7107383608818054], [0, 1]),
 ([0.3691585659980774, 0.6308414340019226], [0, 1]),
 ([0.6567888855934143, 0.3432111442089081], [0, 1]),
 ([0.6573635339736938, 0.34263643622398376], [0, 1]),
 ([0.41456514596939087, 0.5854348540306091], [0, 1]),
 ([0.8514675498008728, 0.1485324651002884], [1, 0]),
 ([0.28460997343063354, 0.7153900265693665], [1, 0]),
 ([0.8218392729759216, 0.17816074192523956], [1, 0]),
 ([0.6419034004211426, 0.3580965995788574], [1, 0]),
 ([0.25269418954849243, 0.7473058104515076], [0, 1]),
 ([0.2668253481388092, 0.7331746816635132], [0, 1]),
 ([0.540794849395752, 0.45920515060424805], [1, 0]),
 ([0.2433258593082428, 0.7566741704940796], [1, 0]),
 ([0.9100537300109863, 0.0899461954832077], [1, 0]),
 ([0.6755228638648987, 0.3244771361351013], [0, 1]),
 ([0.5972504019737244, 0.40274956822395325], [1, 0]),
 ([0.3040631413459778, 0.6959368586540222], [0, 1]),
 ([0.6144446134567261, 0.3855554163455963], [0, 1]),
 ([0.6551833152770996, 0.344816744327545